In [44]:
import json
with open("queries.json", 'r') as f:
    queries = json.load(f)

# for idx, query in enumerate(queries):
# 	print(f"{idx}: {query}")

In [40]:
len(queries)

60

In [90]:
import os
# Function to rename multiple files
def main():
	i = 0
	path="data/raw_data/"
	for filename in os.listdir(path):
		my_dest = 'query_' + str(i) + ".csv"
		my_source = path + filename
		my_dest =path + my_dest
		# rename() function will
		# rename all the files
		os.rename(my_source, my_dest)
		i += 1

main()

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline
# Function to rename multiple files
def clean():
	i = 0
	path="data/raw_data/"
	for filename in os.listdir(path):
		file = 'query_' + str(i) + ".csv"
		df = pd.read_csv(path+file)
		df.columns = [df.iloc[0, 0].split(':')[0].strip()]
		df = df.iloc[1:, :]
		df = df.replace('<1', '0').astype(np.float32)

		# cs = CubicSpline(x = np.arange(0, len(df)*7, step=7), y = df.iloc[:, 0])
		#
		# new_values = []
		# for j in range(7*len(df)):
		# 	new_values.append(cs(j))

		# date_range = pd.date_range(start = df.index[0], periods = len(new_values))
		# data = pd.DataFrame(data = {f'{df.columns[0]}': new_values}, index = date_range)
		# data.to_csv(f'data/clean_data/'+file)

		df.index = pd.to_datetime(df.index)
		df.to_csv(f"data/clean_data/"+file)
		i += 1

clean()

In [2]:
df = pd.read_csv(f"data/clean_data/query_0.csv", index_col = 0)
print(df.index)

Index(['2004-01-01', '2004-02-01', '2004-03-01', '2004-04-01', '2004-05-01',
       '2004-06-01', '2004-07-01', '2004-08-01', '2004-09-01', '2004-10-01',
       ...
       '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01',
       '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01'],
      dtype='object', length=231)


In [3]:
import yfinance as yf
import os
import numpy as np
import pandas as pd

tickers = {
        # 'VGT': "Vanguard Information Technology Index Fund",
        "^TNX": "10Y Bond Yields",
        'DX-Y.NYB': "US Dollar Index",
        '^GSPC': "S&P500 Index",
		"^IRX": "3M Bond Yields"
        }

df = yf.download(list(tickers.keys()), start = '2004-01-01', interval='1mo')['Close']
# df.columns = ['DXY', 'TECH', 'SPX', 'BOND_YIELDS']
# print(df.head())

df['Term Spread'] = df['^TNX'] - df['^IRX']
df.drop(['^IRX'], axis=1, inplace = True)

df1 = pd.read_csv("data/UNRATE.csv", index_col = 0)
df2 = pd.read_csv("data/EXPINF10YR.csv", index_col =0)
df3 = pd.read_csv("data/CPIAUCSL.csv", index_col = 0)
df1.index = pd.to_datetime(df1.index)
df2.index = pd.to_datetime(df2.index)
df3.index = pd.to_datetime(df3.index)

dataframe = pd.concat([df3, df, df1, df2], axis=1, join = 'inner')

mapper = {
		'CPIAUCSL_PC1': 'INFLATION',
		'DX-Y.NYB': 'DXY',
		'^GSPC': 'SP500',
		'UNRATE': 'UNEMPLOYMENT',
		'^TNX': '10Y_BOND'
		}

dataframe = dataframe.rename(columns = mapper)
dataframe = dataframe.ffill()
print(dataframe.head())
dataframe.to_csv('data/control_variables.csv')

[*********************100%***********************]  4 of 4 completed
            INFLATION        DXY        SP500  10Y_BOND  Term Spread  \
2004-01-01    2.02629  87.199997  1131.130005     4.138        3.241   
2004-02-01    1.68845  87.309998  1144.939941     3.984        3.057   
2004-03-01    1.74008  87.610001  1126.209961     3.837        2.914   
2004-04-01    2.29258  90.480003  1107.300049     4.501        3.549   
2004-05-01    2.89776  88.949997  1120.680054     4.655        3.603   

            UNEMPLOYMENT  EXPINF10YR  
2004-01-01           5.7    2.316750  
2004-02-01           5.6    2.275457  
2004-03-01           5.8    2.171686  
2004-04-01           5.6    2.148077  
2004-05-01           5.6    2.454689  


In [4]:
query_data = pd.DataFrame(index = dataframe.index)

i = 0
path="data/clean_data/"
for filename in os.listdir(path):
	file = 'query_' + str(i) + ".csv"
	df = pd.read_csv(path+file, index_col = 0)
	df.index = pd.to_datetime(df.index)
	query_data = pd.concat([query_data, df], axis=1)
	i += 1

print(query_data.info())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 231 entries, 2004-01-01 to 2023-03-01
Freq: MS
Data columns (total 72 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   loan interest calculator                        231 non-null    float64
 1   current interest rate                           231 non-null    float64
 2   home prices                                     231 non-null    float64
 3   Ohio gas prices                                 231 non-null    float64
 4   gas station prices                              231 non-null    float64
 5   house prices                                    231 non-null    float64
 6   current inflation                               231 non-null    float64
 7   costco gas                                      231 non-null    float64
 8   oil prices                                      231 non-null    float64
 9   inflation rates

In [28]:
final_data = pd.concat([dataframe, query_data], axis = 1)
final_data.dropna(inplace = True)
interest_rates=  final_data['interest rates']
final_data = final_data.drop(['interest rates'], axis=1)
final_data['interest rates'] = interest_rates.iloc[:, 0]

print(final_data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 229 entries, 2004-01-01 to 2023-01-01
Freq: MS
Data columns (total 78 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   INFLATION                                       229 non-null    float64
 1   DXY                                             229 non-null    float64
 2   SP500                                           229 non-null    float64
 3   10Y_BOND                                        229 non-null    float64
 4   Term Spread                                     229 non-null    float64
 5   UNEMPLOYMENT                                    229 non-null    float64
 6   EXPINF10YR                                      229 non-null    float64
 7   loan interest calculator                        229 non-null    float64
 8   current interest rate                           229 non-null    float64
 9   home prices    

In [29]:
final_data.to_csv('data/dataset.csv')
final_data.to_stata('data/dataset.dta')

/var/folders/t6/2vvjnfk10mx932fmzp_hb4wr0000gn/T/ipykernel_20491/1647910547.py:2: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    10Y_BOND   ->   _10Y_BOND
    Term Spread   ->   Term_Spread
    loan interest calculator   ->   loan_interest_calculator
    current interest rate   ->   current_interest_rate
    home prices   ->   home_prices
    Ohio gas prices   ->   Ohio_gas_prices
    gas station prices   ->   gas_station_prices
    house prices   ->   house_prices
    current inflation   ->   current_inflation
    costco gas   ->   costco_gas
    oil prices   ->   oil_prices
    inflation rates   ->   inflation_rates
    costco gas prices   ->   costco_gas_prices
    house for sale   ->   house_for_sale
    house sale prices   ->   house_sale_prices
    houses for sale   ->   houses_for_sale
    mortgage rate   ->   mortgage_rate
    gas prices   ->   gas_prices
    natural gas prices   ->   natural_gas_

In [31]:
final_data.columns[final_data.columns.duplicated()]

Index([], dtype='object')